<a href="https://colab.research.google.com/github/ujeongmin/ESAA/blob/main/ESAA_HW_4_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

(파이썬 머신러닝 완벽 가이드)
## [CH8 텍스트 분석]



### 07. 문서 군집화 소개와 실습(Opinion Review 데이터 세트)
#### 문서 군집화 개념
비슷한 텍스트 구성의 문서를 군집화하는 것  
텍스트 분류 기반의 문서 분류는 사전에 결정 카테고리 값을 가진 학습 데이터 세트가 필요한 데 반해, 문서 군집화는 학습 데이터 세트가 필요 없는 비지도학습 기반으로 동작

#### Opinion Review 데이터 세트를 이용한 문서 군집화 수행하기




In [ ]:
import pandas as pd
import glob, os
from google.colab import drive
drive.mount('/content/gdrive')

path = r'/content/gdrive/MyDrive/ESAA/data/topics2'
all_files = glob.glob(os.path.join(path, '*.data'))
filename_list = []
opinion_text = []

for file_ in all_files:
  df = pd.read_table(file_, index_col=None, header=0, encoding='latin1')
  filename_ = file_.split('\\')[-1]
  filename = filename_.split('.')[0]
  filename_list.append(filename)
  opinion_text.append(df.to_string())

document_df = pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head()

Mounted at /content/gdrive


,filename,opinion_text
0,/content/gdrive/MyDrive/ESAA/data/topics2/batt...,...
1,/content/gdrive/MyDrive/ESAA/data/topics2/accu...,...
2,/content/gdrive/MyDrive/ESAA/data/topics2/batt...,...
3,/content/gdrive/MyDrive/ESAA/data/topics2/dire...,...
4,/content/gdrive/MyDrive/ESAA/data/topics2/disp...,...


* 각 파일 이름 자체만으로 의견의 텍스트가 어떠한 제품/서비스에 대한 리뷰인지 알 수 있음
* 문서를 TF-IDF 피처 벡터화

In [ ]:
nltk.download('all')

In [ ]:
# 앞에서 정의한 함수 가져오기

from nltk.stem import WordNetLemmatizer
import nltk
import string

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english',
                             ngram_range=(1,2), min_df=0.05, max_df=0.85)
# opinion_text 칼럼 값으로 피처 벡터화 수행
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


* K-평균을 적용

In [ ]:
from sklearn.cluster import KMeans

# 5개 집합으로 군집화 수행
km_cluster = KMeans(n_clusters=5, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,/content/gdrive/MyDrive/ESAA/data/topics2/batt...,...,1
1,/content/gdrive/MyDrive/ESAA/data/topics2/accu...,...,0
2,/content/gdrive/MyDrive/ESAA/data/topics2/batt...,...,1
3,/content/gdrive/MyDrive/ESAA/data/topics2/dire...,...,0
4,/content/gdrive/MyDrive/ESAA/data/topics2/disp...,...,0


* 파일명으로 정렬해서 군집화 결과 살펴보기

In [ ]:
document_df[document_df['cluster_label']==0].sort_values(by='filename')

,filename,opinion_text,cluster_label
1,/content/gdrive/MyDrive/ESAA/data/topics2/accu...,...,0
3,/content/gdrive/MyDrive/ESAA/data/topics2/dire...,...,0
4,/content/gdrive/MyDrive/ESAA/data/topics2/disp...,...,0
7,/content/gdrive/MyDrive/ESAA/data/topics2/sate...,...,0
8,/content/gdrive/MyDrive/ESAA/data/topics2/scre...,...,0
13,/content/gdrive/MyDrive/ESAA/data/topics2/spee...,...,0
15,/content/gdrive/MyDrive/ESAA/data/topics2/upda...,...,0
17,/content/gdrive/MyDrive/ESAA/data/topics2/voic...,...,0


In [ ]:
document_df[document_df['cluster_label']==1].sort_values(by='filename')

,filename,opinion_text,cluster_label
18,/content/gdrive/MyDrive/ESAA/data/topics2/batt...,...,1
0,/content/gdrive/MyDrive/ESAA/data/topics2/batt...,...,1
2,/content/gdrive/MyDrive/ESAA/data/topics2/batt...,...,1
6,/content/gdrive/MyDrive/ESAA/data/topics2/keyb...,...,1
29,/content/gdrive/MyDrive/ESAA/data/topics2/perf...,...,1
9,/content/gdrive/MyDrive/ESAA/data/topics2/scre...,...,1
10,/content/gdrive/MyDrive/ESAA/data/topics2/scre...,...,1
11,/content/gdrive/MyDrive/ESAA/data/topics2/size...,...,1
12,/content/gdrive/MyDrive/ESAA/data/topics2/soun...,headphone jack i got a clear case for it a...,1
16,/content/gdrive/MyDrive/ESAA/data/topics2/vide...,...,1


In [ ]:
document_df[document_df['cluster_label']==2].sort_values(by='filename')

,filename,opinion_text,cluster_label
37,/content/gdrive/MyDrive/ESAA/data/topics2/bath...,...,2
23,/content/gdrive/MyDrive/ESAA/data/topics2/food...,...,2
24,/content/gdrive/MyDrive/ESAA/data/topics2/food...,...,2
39,/content/gdrive/MyDrive/ESAA/data/topics2/free...,...,2
41,/content/gdrive/MyDrive/ESAA/data/topics2/loca...,...,2
27,/content/gdrive/MyDrive/ESAA/data/topics2/loca...,...,2
43,/content/gdrive/MyDrive/ESAA/data/topics2/park...,...,2
31,/content/gdrive/MyDrive/ESAA/data/topics2/pric...,...,2
32,/content/gdrive/MyDrive/ESAA/data/topics2/room...,...,2
46,/content/gdrive/MyDrive/ESAA/data/topics2/room...,...,2


In [ ]:
document_df[document_df['cluster_label']==3].sort_values(by='filename')

,filename,opinion_text,cluster_label
19,/content/gdrive/MyDrive/ESAA/data/topics2/butt...,...,3
21,/content/gdrive/MyDrive/ESAA/data/topics2/eyes...,...,3
5,/content/gdrive/MyDrive/ESAA/data/topics2/feat...,...,3
22,/content/gdrive/MyDrive/ESAA/data/topics2/font...,...,3
28,/content/gdrive/MyDrive/ESAA/data/topics2/navi...,...,3
30,/content/gdrive/MyDrive/ESAA/data/topics2/pric...,...,3
14,/content/gdrive/MyDrive/ESAA/data/topics2/spee...,...,3


In [ ]:
document_df[document_df['cluster_label']==4].sort_values(by='filename')

,filename,opinion_text,cluster_label
38,/content/gdrive/MyDrive/ESAA/data/topics2/comf...,...,4
20,/content/gdrive/MyDrive/ESAA/data/topics2/comf...,...,4
25,/content/gdrive/MyDrive/ESAA/data/topics2/gas_...,...,4
40,/content/gdrive/MyDrive/ESAA/data/topics2/inte...,...,4
26,/content/gdrive/MyDrive/ESAA/data/topics2/inte...,...,4
42,/content/gdrive/MyDrive/ESAA/data/topics2/mile...,...,4
44,/content/gdrive/MyDrive/ESAA/data/topics2/perf...,...,4
45,/content/gdrive/MyDrive/ESAA/data/topics2/qual...,...,4
47,/content/gdrive/MyDrive/ESAA/data/topics2/seat...,...,4
50,/content/gdrive/MyDrive/ESAA/data/topics2/tran...,...,4


* 파일명에 따라 주제를 살펴보면 군집 개수가 약간 많아 세분화되어 군집화된 경향이 있음, 3개 군집으로 낮추기

In [ ]:
# 3개 집합으로 군집화 수행
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

# 소속 군집을 cluster_label 칼럼으로 할당하고 cluster_label 값으로 정렬
document_df['cluster_label'] = cluster_label
document_df.sort_values(by='cluster_label')

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,filename,opinion_text,cluster_label
0,/content/gdrive/MyDrive/ESAA/data/topics2/batt...,...,0
30,/content/gdrive/MyDrive/ESAA/data/topics2/pric...,...,0
29,/content/gdrive/MyDrive/ESAA/data/topics2/perf...,...,0
28,/content/gdrive/MyDrive/ESAA/data/topics2/navi...,...,0
22,/content/gdrive/MyDrive/ESAA/data/topics2/font...,...,0
19,/content/gdrive/MyDrive/ESAA/data/topics2/butt...,...,0
18,/content/gdrive/MyDrive/ESAA/data/topics2/batt...,...,0
17,/content/gdrive/MyDrive/ESAA/data/topics2/voic...,...,0
16,/content/gdrive/MyDrive/ESAA/data/topics2/vide...,...,0
15,/content/gdrive/MyDrive/ESAA/data/topics2/upda...,...,0


In [ ]:
document_df[document_df['cluster_label']==0].sort_values(by='filename')

,filename,opinion_text,cluster_label
1,/content/gdrive/MyDrive/ESAA/data/topics2/accu...,...,0
18,/content/gdrive/MyDrive/ESAA/data/topics2/batt...,...,0
0,/content/gdrive/MyDrive/ESAA/data/topics2/batt...,...,0
2,/content/gdrive/MyDrive/ESAA/data/topics2/batt...,...,0
19,/content/gdrive/MyDrive/ESAA/data/topics2/butt...,...,0
3,/content/gdrive/MyDrive/ESAA/data/topics2/dire...,...,0
4,/content/gdrive/MyDrive/ESAA/data/topics2/disp...,...,0
21,/content/gdrive/MyDrive/ESAA/data/topics2/eyes...,...,0
5,/content/gdrive/MyDrive/ESAA/data/topics2/feat...,...,0
22,/content/gdrive/MyDrive/ESAA/data/topics2/font...,...,0


In [ ]:
document_df[document_df['cluster_label']==1].sort_values(by='filename')

,filename,opinion_text,cluster_label
37,/content/gdrive/MyDrive/ESAA/data/topics2/bath...,...,1
23,/content/gdrive/MyDrive/ESAA/data/topics2/food...,...,1
24,/content/gdrive/MyDrive/ESAA/data/topics2/food...,...,1
39,/content/gdrive/MyDrive/ESAA/data/topics2/free...,...,1
41,/content/gdrive/MyDrive/ESAA/data/topics2/loca...,...,1
27,/content/gdrive/MyDrive/ESAA/data/topics2/loca...,...,1
43,/content/gdrive/MyDrive/ESAA/data/topics2/park...,...,1
31,/content/gdrive/MyDrive/ESAA/data/topics2/pric...,...,1
32,/content/gdrive/MyDrive/ESAA/data/topics2/room...,...,1
46,/content/gdrive/MyDrive/ESAA/data/topics2/room...,...,1


In [ ]:
document_df[document_df['cluster_label']==2].sort_values(by='filename')

,filename,opinion_text,cluster_label
38,/content/gdrive/MyDrive/ESAA/data/topics2/comf...,...,2
20,/content/gdrive/MyDrive/ESAA/data/topics2/comf...,...,2
25,/content/gdrive/MyDrive/ESAA/data/topics2/gas_...,...,2
40,/content/gdrive/MyDrive/ESAA/data/topics2/inte...,...,2
26,/content/gdrive/MyDrive/ESAA/data/topics2/inte...,...,2
42,/content/gdrive/MyDrive/ESAA/data/topics2/mile...,...,2
44,/content/gdrive/MyDrive/ESAA/data/topics2/perf...,...,2
45,/content/gdrive/MyDrive/ESAA/data/topics2/qual...,...,2
47,/content/gdrive/MyDrive/ESAA/data/topics2/seat...,...,2
50,/content/gdrive/MyDrive/ESAA/data/topics2/tran...,...,2


* Cluster #0은 포터블 전자기기 리뷰, #1은 호텔 리뷰, #2는 자동차 리뷰로 군집화가 잘 구성

#### 군집별 핵심 단어 추출하기
cluster_centers_는 배열 값으로 제공되며, 행은 개별 군집 열은 개별 피처를 의미  
각 배열 내의 값은 개별 군집내의 상대 위치를 숫자 값으로 표현한 일종의 좌표 값

In [ ]:
cluster_centers = km_cluster.cluster_centers_
print('cluster_centers shape :', cluster_centers.shape)
print(cluster_centers)

cluster_centers shape : (3, 4611)
[[0.01005322 0.         0.         ... 0.00706287 0.         0.        ]
 [0.         0.00099499 0.00174637 ... 0.         0.00183397 0.00144581]
 [0.         0.00092551 0.         ... 0.         0.         0.        ]]


* 군집이 3개, word 피처가 4611개로 구성되어있음, 1에 가까울수록 중심과 가까운 값


* argsort()[:,::-1] 이용하면 배열 내 값이 큰 순으로 정렬된 위치 인덱스 값을 반환 (정렬된 값이 아니라 인덱스)
* get_cluster_details() 함수를 생성해 각 군집별 핵심 단어 찾기

In [ ]:
# 군집별 top n 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명을 반환함
def get_cluster_details(cluster_model, cluster_data, feature_names,
                        clusters_num, top_n_features=10):
  cluster_details = {}

  # cluster_centers array의 값이 큰 순으로 정렬된 인덱스 값을 반환
  # 군집 중심점(centroid)별 할당된 word 피처들의 거리값이 큰 순으로 값을 구하기 위함
  centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:,::-1]

  # 개별 군집별로 반복하면서 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명 입력
  for cluster_num in range(clusters_num):
    # 개별 군집별 정보를 담을 데이터 초기화
    cluster_details[cluster_num] = {}
    cluster_details[cluster_num]['cluster'] = cluster_num

    # cluster_centers_.argsort()[:, ::-1]로 구한 인덱스를 이용해 top n 피처 단어를 구함
    top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
    top_features = [feature_names[ind] for ind in top_feature_indexes]

    # top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함
    top_feature_values = cluster_model.cluster_centers_[cluster_num,
                                                        top_feature_indexes].tolist()

    # cluster_details 딕셔너리 객체에 개별 군집별 핵심단어와 중심위치 상댓값, 해당 파일명 입력
    cluster_details[cluster_num]['top_features'] = top_features
    cluster_details[cluster_num]['top_features_value'] = top_feature_values
    filenames = cluster_data[cluster_data['cluster_label'] == cluster_num]['filename']
    filenames = filenames.values.tolist()

    cluster_details[cluster_num]['filenames'] = filenames

  return cluster_details

* get_cluster_details()를 호출하면 딕셔너리를 원소로 가지는 리스트인 cluster_details를 반환
* 개별 군집번호, 핵심 단어, 핵심단어 중심 위치 상댓값, 파일명 속성 값 정보가 들어있는데 보기 좋게 출력하기 위해 별도의 print_cluster_details() 함수 생성

In [ ]:
def print_cluster_details(cluster_details):
  for cluster_num, cluster_detail in cluster_details.items():
    print('###### Cluster {0}'.format(cluster_num))
    print('Top features :', cluster_detail['top_features'])
    print('Reviews 파일명 :', cluster_detail['filenames'][:7])
    print('==============================================')

In [ ]:
feature_names = tfidf_vect.get_feature_names_out()

cluster_details = get_cluster_details(cluster_model=km_cluster, cluster_data=document_df,
                                      feature_names=feature_names, clusters_num=3, top_n_features=10)
print_cluster_details(cluster_details)

###### Cluster 0
Top features : ['screen', 'battery', 'keyboard', 'battery life', 'life', 'kindle', 'direction', 'video', 'size', 'voice']
Reviews 파일명 : ['/content/gdrive/MyDrive/ESAA/data/topics2/battery-life_ipod_nano_8gb', '/content/gdrive/MyDrive/ESAA/data/topics2/accuracy_garmin_nuvi_255W_gps', '/content/gdrive/MyDrive/ESAA/data/topics2/battery-life_netbook_1005ha', '/content/gdrive/MyDrive/ESAA/data/topics2/directions_garmin_nuvi_255W_gps', '/content/gdrive/MyDrive/ESAA/data/topics2/display_garmin_nuvi_255W_gps', '/content/gdrive/MyDrive/ESAA/data/topics2/features_windows7', '/content/gdrive/MyDrive/ESAA/data/topics2/keyboard_netbook_1005ha']
###### Cluster 1
Top features : ['room', 'hotel', 'service', 'staff', 'food', 'location', 'bathroom', 'clean', 'price', 'parking']
Reviews 파일명 : ['/content/gdrive/MyDrive/ESAA/data/topics2/food_holiday_inn_london', '/content/gdrive/MyDrive/ESAA/data/topics2/food_swissotel_chicago', '/content/gdrive/MyDrive/ESAA/data/topics2/location_holiday_

* Cluster #0에서는 화면과 배터리 수명 등이, #1에서는 방과 서비스 등이, #2에서는 실내 인테리어, 좌석, 연료 효율 등이 핵심 단어로 군집화됨